# AgentChat - First Assistant Agent

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [2]:
model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)

In [4]:
assistant = AssistantAgent(name='assistant',model_client=model_client)


In [5]:
result = await assistant.run(task="What is the capital of France?")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the capital of France?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=7), metadata={}, content='The capital of France is Paris.', type='TextMessage')] stop_reason=None


In [ ]:
assistant_2 = AssistantAgent(name='assistant',
                           model_client=model_client,
                           description="Give output in JSON",
                           system_message="You are a helpful assistant that provides accurate information about history.Give output in JSON"
                           )


In [12]:
result = await assistant_2.run(task="What is the capital of France?")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the capital of France?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=35, completion_tokens=20), metadata={}, content='```json\n{\n  "country": "France",\n  "capital": "Paris"\n}\n```', type='TextMessage')] stop_reason=None


In [13]:
result.messages[-1].content

'```json\n{\n  "country": "France",\n  "capital": "Paris"\n}\n```'

In [14]:
result = await assistant_2.run(task="What I asked you in the last question?")
result.messages[-1].content

'```json\n{\n  "previous_question": "What is the capital of France?"\n}\n```'

In [ ]:
# Prompt Engineering

In [24]:
from io import BytesIO

import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image

pil_image = Image.open(BytesIO(requests.get("https://picsum.photos/id/9/200/400").content))
img = AGImage(pil_image)
multi_modal_message = MultiModalMessage(content=["Can you describe the content of this image?", img], source="User")


In [23]:
multi_model_message = MultiModalMessage(
    content = ['What is there in the image?',img],
    source="User"
)

result = await assistant_2.run(task=multi_model_message)
result.messages[-1].content

'```json\n{\n  "image_content": "A laptop on a wooden surface."\n}\n```'

In [25]:
def get_weather(city:str)->str:
    # Simulate a weather API call
    return f"The weather in {city} is sunny with a temperature of 25°C."


weather_agent = AssistantAgent(
    name='weather_agent',
    model_client=model_client,
    description="You are a weather assistant that provides accurate weather information.",
    system_message="You are a helpful assistant that provides accurate weather information by using the get_weather.",
    tools=[get_weather]
)

result = await weather_agent.run(task="What is the weather in Delhi?")
result.messages[-1].content


'The weather in Delhi is sunny with a temperature of 25°C.'

# Strucutred Output

In [ ]:
from pydantic import BaseModel

class Information(BaseModel):
    fact:str
    source:str


model_client_structured = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)

structure_output_agent = AssistantAgent(
    name='assistant',
    model_client=model_client_structured,
    description="You are a helpful assistant that provides accurate information about history in Structure Response.",
    output_content_type=Information,
    system_message="You are a helpful assistant that provides accurate information about history.",
)

result = await structure_output_agent.run(task="What is the capital of India?")
print(result.messages[-1].content)

{"fact":"The capital of India is New Delhi.","source":"General knowledge and encyclopedic sources."}


In [ ]:
result.message

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the capital of India?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=21), metadata={}, content='{"fact":"The capital of India is New Delhi.","source":"General knowledge and encyclopedic sources."}', type='TextMessage')], stop_reason=None)

In [35]:
structure_output_agent.model_context.get_messages

<bound method UnboundedChatCompletionContext.get_messages of <autogen_core.model_context._unbounded_chat_completion_context.UnboundedChatCompletionContext object at 0x116fee510>>